In [ ]:
pip install datasets

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, TaskType


In [ ]:

# Step 1: Load tokenizer and dataset
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

dataset = load_dataset("ag_news")
dataset = dataset.map(lambda e: tokenizer(e["text"], truncation=True, padding="max_length"), batched=True)
dataset = dataset.rename_column("label", "labels")
dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

# Step 2: Load pre-trained model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=4)

# Step 3: Define LoRA configuration
lora_config = LoraConfig(
    r=8,                        # Low-rank size
    lora_alpha=16,              # Scaling factor
    target_modules=["query", "value"],  # Apply LoRA to attention Q and V
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_CLS  # Sequence classification task
)

# Step 4: Apply LoRA to the model
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# Step 5: Define training arguments
training_args = TrainingArguments(
    output_dir="./results-lora-agnews",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    learning_rate=2e-4,
    report_to="none"
)

# Step 6: Train using Trainer API
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"].select(range(2000)),  # smaller subset for speed
    eval_dataset=dataset["test"].select(range(500)),
)

trainer.train()
